In [1]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [2]:
train_df.shape

(256351, 69)

## columns drop

In [3]:
# 컬럼 삭제
columns_to_drop = [
    "임신 시도 또는 마지막 임신 경과 연수",
    "착상 전 유전 검사 사용 여부",
    "PGD 시술 여부",
    "PGS 시술 여부",
    "난자 해동 경과일",
    "배아 해동 경과일",
    "불임 원인 - 여성 요인",
    "난자 채취 경과일",
    "배란 유도 유형",
    "불임 원인 - 정자 면역학적 요인"
]

train_df.drop(columns=columns_to_drop, inplace=True)
print(train_df.shape)

(256351, 59)


## 결측치 처리

In [4]:
# 특정 시술 때문에 없는 값은 -1로 처리
columns_to_fill_na = [
    "단일 배아 이식 여부", "착상 전 유전 진단 사용 여부", "배아 생성 주요 이유", "총 생성 배아 수",
    "미세주입된 난자 수", "미세주입에서 생성된 배아 수", "이식된 배아 수", "미세주입 배아 이식 수",
    "저장된 배아 수", "미세주입 후 저장된 배아 수", "해동된 배아 수", "해동 난자 수", 
    "수집된 신선 난자 수", "저장된 신선 난자 수", "혼합된 난자 수", "파트너 정자와 혼합된 난자 수", 
    "기증자 정자와 혼합된 난자 수", "동결 배아 사용 여부", "신선 배아 사용 여부", "기증 배아 사용 여부", "대리모 여부"
]

train_df[columns_to_fill_na] = train_df[columns_to_fill_na].fillna(-1)

In [5]:
columns_to_fill = ["난자 혼합 경과일", "배아 이식 경과일"]

# "DI" 시술인 경우 -1로 대체
train_df.loc[train_df["시술 유형"] == "DI", columns_to_fill] = -1

# "IVF" 시술인 경우 중앙값으로 대체
train_df.loc[(train_df["시술 유형"] == "IVF") & (train_df["난자 혼합 경과일"].isnull()), "난자 혼합 경과일"] = 0.0
train_df.loc[(train_df["시술 유형"] == "IVF") & (train_df["배아 이식 경과일"].isnull()), "배아 이식 경과일"] = 3.0


# '배아 이식 경과일' 중앙값: 3.0 , 최빈값: 5.0입니다.
# 최빈값으로 대체하고 싶으신 경우 5로 바꿔주시면 됩니다. 


In [6]:
# 적은 값이기 때문에 행 삭제
train_df = train_df.dropna(subset=["특정 시술 유형"])

## 로그 변환

In [7]:
# 해당 없음을 표현한 -1값을 제외하고 로그변환
import numpy as np

columns_to_log_transform = [
    "총 생성 배아 수", "미세주입된 난자 수", "미세주입에서 생성된 배아 수", "저장된 배아 수", 
    "기증자 정자와 혼합된 난자 수", "파트너 정자와 혼합된 난자 수", "혼합된 난자 수", 
    "저장된 신선 난자 수", "수집된 신선 난자 수", "해동 난자 수", 
    "해동된 배아 수", "미세주입 후 저장된 배아 수", "이식된 배아 수", "미세주입 배아 이식 수"
]

for col in columns_to_log_transform:
    mask = train_df[col] != -1 
    train_df.loc[mask, col] = np.log1p(train_df.loc[mask, col])

## 데이터 타입 변경

In [8]:
# 범주형 컬럼 string으로 변경
categorical_columns = [
    "시술 시기 코드", "시술 당시 나이", "시술 유형", "특정 시술 유형", "배아 생성 주요 이유", 
    "총 시술 횟수", "클리닉 내 총 시술 횟수", "IVF 시술 횟수", "DI 시술 횟수", 
    "총 임신 횟수", "IVF 임신 횟수", "DI 임신 횟수", "총 출산 횟수", 
    "IVF 출산 횟수", "DI 출산 횟수", "난자 출처", "정자 출처", 
    "난자 기증자 나이", "정자 기증자 나이"
]

train_df[categorical_columns] = train_df[categorical_columns].astype(str)

In [9]:
# 이진형 변수, 경과일 컬럼 int형으로 변경
binary_columns = [
    "배란 자극 여부", "단일 배아 이식 여부", "착상 전 유전 진단 사용 여부", "남성 주 불임 원인", 
    "남성 부 불임 원인", "여성 주 불임 원인", "여성 부 불임 원인", "부부 주 불임 원인", 
    "부부 부 불임 원인", "불명확 불임 원인", "불임 원인 - 난관 질환", "불임 원인 - 남성 요인", 
    "불임 원인 - 배란 장애", "불임 원인 - 자궁경부 문제", "불임 원인 - 자궁내막증", "불임 원인 - 정자 농도", 
    "불임 원인 - 정자 운동성", "불임 원인 - 정자 형태", "동결 배아 사용 여부", "신선 배아 사용 여부", 
    "기증 배아 사용 여부", "대리모 여부", "난자 혼합 경과일", "배아 이식 경과일"
]

train_df[binary_columns] = train_df[binary_columns].astype(int)

In [10]:
# 로그 변환을 한 컬럼들 float형으로 변경
numeric_columns = [
    "총 생성 배아 수", "미세주입된 난자 수", "미세주입에서 생성된 배아 수", "저장된 배아 수", 
    "기증자 정자와 혼합된 난자 수", "파트너 정자와 혼합된 난자 수", "혼합된 난자 수", 
    "저장된 신선 난자 수", "수집된 신선 난자 수", "해동 난자 수", 
    "해동된 배아 수", "미세주입 후 저장된 배아 수", "이식된 배아 수", "미세주입 배아 이식 수" 
]

train_df[numeric_columns] = train_df[numeric_columns].astype(float)

## 데이터 확인

In [11]:
print(f"데이터 크기: {train_df.shape}")

데이터 크기: (256349, 59)


In [12]:
train_df.head()

,ID,시술 시기 코드,시술 당시 나이,시술 유형,특정 시술 유형,배란 자극 여부,단일 배아 이식 여부,착상 전 유전 진단 사용 여부,남성 주 불임 원인,남성 부 불임 원인,...,정자 출처,난자 기증자 나이,정자 기증자 나이,동결 배아 사용 여부,신선 배아 사용 여부,기증 배아 사용 여부,대리모 여부,난자 혼합 경과일,배아 이식 경과일,임신 성공 여부
0,TRAIN_000000,TRZKPL,만18-34세,IVF,ICSI,1,0,0,0,0,...,배우자 제공,알 수 없음,알 수 없음,0,1,0,0,0,3,0
1,TRAIN_000001,TRYBLT,만45-50세,IVF,ICSI,0,0,0,0,0,...,배우자 제공,알 수 없음,알 수 없음,0,1,0,0,0,3,0
2,TRAIN_000002,TRVNRY,만18-34세,IVF,IVF,1,0,0,0,0,...,배우자 제공,알 수 없음,알 수 없음,0,1,0,0,0,2,0
3,TRAIN_000003,TRJXFG,만35-37세,IVF,ICSI,1,0,0,0,0,...,배우자 제공,알 수 없음,알 수 없음,0,1,0,0,0,3,0
4,TRAIN_000004,TRVNRY,만18-34세,IVF,ICSI,1,0,0,0,0,...,배우자 제공,알 수 없음,알 수 없음,0,1,0,0,0,3,0
